In [ ]:
# Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

Medical_category = '4bf58dd8d48988d104941735' # 'Root' category for all food-related venues

Medical_Services_subcategories = ['52e81612bcbc57f1066b7a3b','52e81612bcbc57f1066b7a3c','52e81612bcbc57f1066b7a3a',
                                 '4bf58dd8d48988d178941735','4bf58dd8d48988d177941735','4bf58dd8d48988d194941735',
                                 '522e32fae4b09b556e370f19','4bf58dd8d48988d196941735','58daa1558bbb0b01f18ec1f7',
                                 '56aa371be4b08b9a8d5734ff','4f4531b14b9074f6e4fb0103','52e81612bcbc57f1066b7a39',
                                 '58daa1558bbb0b01f18ec1d0','5744ccdfe4b0c0459246b4af','56aa371be4b08b9a8d57351d',
                                 '56aa371be4b08b9a8d573526']

def is_medical(categories, specific_filter=None):
    medical_words = ['clinica', 'consultorio', 'laboratorio', 'hospital', 'maternidade']
    medical = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in medical_words:
            if r in category_name:
                medical = True
            if not(specific_filter is None) and (category_id in specific_filter):
                specific = True
                medical = True
    return medical, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Recife', '')
    address = address.replace(', PE', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
        
    return venues

In [ ]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found italian restaurants

import pickle

def get_medical(lats, lons):
    medical_service = {}
    sub_medical = {}
    location_centers = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        area_medical = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_med, is_serv = is_medical(venue_categories, specific_filter=Medical_Services_subcategories)
            if is_med:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                medical_service = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_serv, x, y)
                if venue_distance<=300:
                    area_restaurants.append(medical_service)
                medical_service[venue_id] = medical_service
                if is_serv:
                    sub_medical[venue_id] = medical_service
        location_centers.append(area_medical)
        print(' .', end='')
    print(' done.')
    return medical_service, sub_medical, location_centers

# Try to load from local file system in case we did this before
medical_service = {}
sub_medical = {}
location_centers = []
loaded = False
try:
    with open('medical_service.pkl', 'rb') as f:
        medical_services = pickle.load(f)
    with open('sub_medical.pkl', 'rb') as f:
        sub_medical = pickle.load(f)
    with open('location_centers.pkl', 'rb') as f:
        location_centers = pickle.load(f)
    print('Medical data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    medical_service, sub_medical, location_centers = get_medical(lat, lon)
    
    # Let's persists this in local file system
    with open('medical_service.pkl', 'wb') as f:
        pickle.dump(medical_services, f)
    with open('sub_medical.pkl', 'wb') as f:
        pickle.dump(sub_medical, f)
    with open('location_centers.pkl', 'wb') as f:
        pickle.dump(location_centers, f)
        